## Data Extraction & Scraping

This project aims to construct a comprehensive, "ready-to-use" dataset of European football matches enriched with geospatial data. By combining match schedules with precise stadium coordinates, we enable location-based analytics (e.g., finding accommodation near a specific match).

The project follows a robust **ETL (Extract, Transform, Load)** architecture:

- **Extract**: We ingest raw data from web scraping (Wikipedia) and open data repositories (CSV files).
- **Transform**: We clean, normalize, and map disparate data sources to a common schema.
- **Load**: We enrich the data and store it in a unified table for downstream usage.

#### Infrastructure & Proxy Configuration

To perform web scraping at scale without triggering anti-bot mechanisms or IP bans, we utilize the **Bright Data** proxy network. This allows us to route our requests through legitimate IP addresses.

We configure our connection using the **Native access via proxy user name and password** method. We control the proxy behavior dynamically by modifying the username string in our code

### 📍 A. Stadium Geolocation (Web Scraping)
- **Source**: Wikipedia (16 European Domestic Leagues).
- **Goal**: To acquire precise GPS coordinates (Latitude & Longitude) for over 150 European stadiums.
- **Method**: We utilize `BeautifulSoup` to navigate Wikipedia's HTML structure, extracting hidden geographical tags. The pipeline includes a **Dynamic Column Detection** system to automatically identify data columns (e.g., *Team* vs *Stadium*) by analyzing headers and keyword density where fixed indexing would fail.

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import re
import random
import urllib3
import warnings
import time

# ==========================================
# 0. SETUP & CONFIGURATION
# ==========================================
# Suppress SSL warnings to keep output clean
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.filterwarnings("ignore")

# Proxy Configuration (User Provided)
PASSWORD = os.getenv('BRIGHTDATA_PASSWORD', 'YOUR_PASSWORD_HERE')
BASE_USERNAME = os.getenv('BRIGHTDATA_USER', 'YOUR_ZONE_USERNAME_HERE') 
HOST = 'brd.superproxy.io'
PORT = '33335'
country_code = "hu" 
session_id = random.randint(1, 1000000)
FINAL_USERNAME = f"{BASE_USERNAME}-country-{country_code}-session-{session_id}"

PROXIES = {
    'http': f'http://{FINAL_USERNAME}:{PASSWORD}@{HOST}:{PORT}', 
    'https': f'http://{FINAL_USERNAME}:{PASSWORD}@{HOST}:{PORT}'
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/120.0.0.0 Safari/537.36'
}

# Supported Wiki languages for fallback coordinate search
WIKI_LANGUAGES = ['en', 'de', 'fr', 'es', 'it', 'pt', 'nl', 'tr', 'pl', 'el', 'cs', 'da', 'hr', 'uk']

# ==========================================
# 1. ROBUST HELPER FUNCTIONS
# ==========================================

def safe_int(val, default=1):
    """Safely converts a value to int. Returns default if None or invalid."""
    if val is None:
        return default
    try:
        # Handles strings like "2", "2.0", or integers
        return int(float(val))
    except (ValueError, TypeError):
        return default

def dms_to_decimal(dms_str):
    """Converts Degrees-Minutes-Seconds string to decimal latitude/longitude."""
    try:
        if not dms_str: return None
        dms_str = dms_str.strip().upper()
        # Split by degree/minute/second symbols
        parts = re.split(r'[°′\'″"]+', dms_str)
        if len(parts) < 3: return None
        
        deg = float(parts[0])
        mn = float(parts[1]) if parts[1] else 0.0
        # Handle seconds and direction
        sec = 0.0
        direction = 'N'
        
        # Logic to find seconds and direction (N/S/E/W)
        if len(parts) > 2 and parts[2]:
            if parts[2] in ['N','S','E','W']:
                direction = parts[2]
            else:
                sec = float(parts[2])
                if len(parts) > 3 and parts[3] in ['N','S','E','W']:
                    direction = parts[3]
        
        # Check last character of original string for direction if not found yet
        if direction == 'N' and dms_str[-1] in ['N','S','E','W']:
            direction = dms_str[-1]

        val = deg + (mn / 60.0) + (sec / 3600.0)
        return -val if direction in ['S', 'W'] else val
    except: 
        return None

def get_interlanguage_links(url):
    """Finds links to the same Wikipedia article in other languages."""
    try:
        r = requests.get(url, proxies=PROXIES, headers=HEADERS, verify=False, timeout=10)
        if r.status_code != 200: return {}
        soup = BeautifulSoup(r.content, 'html.parser')
        
        lang_links = {}
        for link in soup.find_all('a', class_='interlanguage-link-target'):
            href = link.get('href', '')
            match = re.search(r'https://(\w+)\.wikipedia\.org', href)
            if match:
                lang_links[match.group(1)] = href
        return lang_links
    except:
        return {}

def try_coords_from_url(url):
    """Attempts to extract coordinates from a single Wikipedia URL."""
    try:
        r = requests.get(url, proxies=PROXIES, headers=HEADERS, verify=False, timeout=10)
        if r.status_code != 200: return None, None
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # Method 1: Standard Geo tag (decimal)
        geo = soup.find('span', class_='geo')
        if geo:
            try:
                parts = geo.text.split(';')
                return float(parts[0]), float(parts[1])
            except: pass
            
        # Method 2: Latitude/Longitude spans (DMS)
        lat_elem = soup.find('span', class_='latitude')
        lon_elem = soup.find('span', class_='longitude')
        if lat_elem and lon_elem:
            return dms_to_decimal(lat_elem.text), dms_to_decimal(lon_elem.text)
    except:
        pass
    return None, None

def get_coords(url):
    """
    Main coordinate fetcher. 
    1. Tries the English page.
    2. If missing, tries other language versions (German, French, etc.) of the same page.
    """
    if not url or "redlink=1" in url: return None, None
    if url.startswith('/'): url = f"https://en.wikipedia.org{url}"
    
    # Try 1: Direct URL
    lat, lon = try_coords_from_url(url)
    if lat: return lat, lon
    
    # Try 2: Interlanguage links
    lang_links = get_interlanguage_links(url)
    for lang in WIKI_LANGUAGES:
        if lang == 'en': continue
        
        alt_url = lang_links.get(lang)
        if not alt_url:
            # Construct guessed URL if link not found
            title = url.split('/wiki/')[-1]
            alt_url = f"https://{lang}.wikipedia.org/wiki/{title}"
        
        lat, lon = try_coords_from_url(alt_url)
        if lat: return lat, lon
        time.sleep(0.5) # Be polite
    
    return None, None

def parse_html_table_to_matrix(table):
    """
    Parses an HTML table into a 2D list (matrix), handling rowspans and colspans.
    Uses safe_int() to prevent TypeErrors.
    """
    rows = table.find_all('tr')
    # Initialize a large empty grid
    matrix = []
    for _ in rows: matrix.append([None] * 50) 
    
    for r_idx, row in enumerate(rows):
        cells = row.find_all(['td', 'th'])
        c_idx = 0
        for cell in cells:
            # Skip cells that are already filled by a previous rowspan
            while matrix[r_idx][c_idx] is not None: 
                c_idx += 1
            
            # Extract text and link
            text = cell.get_text(strip=True).split('\n')[0]
            link_tag = cell.find('a')
            link = link_tag.get('href') if link_tag else None
            
            cell_data = {'text': text, 'link': link}
            
            # --- FIXED LOGIC HERE ---
            # Safely get rowspan and colspan
            rs = safe_int(cell.get('rowspan'), default=1)
            cs = safe_int(cell.get('colspan'), default=1)
            
            # Fill the matrix
            for r in range(rs):
                for c in range(cs):
                    if r_idx + r < len(matrix):
                        matrix[r_idx + r][c_idx + c] = cell_data
            
            c_idx += cs
            
    # Remove empty trailing columns/rows
    return [row for row in matrix if any(c is not None for c in row)]

def find_columns(matrix):
    """Identifies the index of the 'Team' and 'Stadium' columns."""
    if not matrix or len(matrix) < 2: return -1, -1
    
    headers = [c['text'].lower() if c else '' for c in matrix[0]]
    t_idx, s_idx = -1, -1
    
    # 1. Try finding specific Headers
    for i, h in enumerate(headers):
        if any(x in h for x in ['team', 'club']): t_idx = i
        if any(x in h for x in ['stadium', 'venue', 'ground', 'arena']): s_idx = i
        
    if t_idx != -1 and s_idx != -1 and t_idx != s_idx:
        return t_idx, s_idx

    # 2. Fallback: Scan column content for keywords
    stadium_keywords = ['stadium', 'arena', 'park', 'field', 'ground', 'stadion']
    max_cols = max(len(r) for r in matrix)
    s_scores = [0] * max_cols
    
    for c in range(max_cols):
        col_texts = [r[c]['text'].lower() for r in matrix[1:] if c < len(r) and r[c]]
        s_scores[c] = sum(1 for t in col_texts if any(k in t for k in stadium_keywords))
    
    if max(s_scores) > 0:
        s_idx = s_scores.index(max(s_scores))
        
    # Heuristic: If we found stadium, Team is usually col 0 or 1
    if s_idx != -1:
        if s_idx == 0: t_idx = 1
        else: t_idx = 0
        
    return t_idx, s_idx

# ==========================================
# 2. MAIN LOGIC
# ==========================================
def scrape_league(league_name, country_code, url):
    print(f"Scraping: {league_name}...")
    data = []
    
    try:
        r = requests.get(url, proxies=PROXIES, headers=HEADERS, verify=False, timeout=20)
        soup = BeautifulSoup(r.content, 'html.parser')
        tables = soup.find_all('table', class_=['wikitable', 'article-table'])
        
        for table in tables:
            matrix = parse_html_table_to_matrix(table)
            t_idx, s_idx = find_columns(matrix)
            
            if t_idx == -1 or s_idx == -1: continue
            
            table_data = []
            valid_table = True
            
            for row in matrix[1:]:
                # Ensure row has enough columns
                if len(row) <= max(t_idx, s_idx): continue
                
                t_cell = row[t_idx]
                s_cell = row[s_idx]
                
                if not t_cell or not s_cell: continue
                
                t_text = t_cell['text']
                s_text = s_cell['text']
                
                # Validation: Skip "Capacity" or "Location" rows that look like headers
                if len(t_text) < 2 or len(s_text) < 3: continue
                if any(x in t_text.lower() for x in ['stadium', 'capacity', 'location']):
                    valid_table = False
                    break
                    
                # Get Coordinates
                lat, lon = get_coords(s_cell['link'])
                
                table_data.append({
                    "League": league_name,
                    "Country": country_code,
                    "Team": t_text,
                    "Stadium": s_text,
                    "Latitude": lat,
                    "Longitude": lon
                })
            
            # If table yielded good data, we are done with this league
            if valid_table and len(table_data) >= 4:
                data.extend(table_data)
                found = sum(1 for d in table_data if d['Latitude'])
                print(f"   -> Success. Found {len(table_data)} teams. ({found} with coords)")
                break
                
    except Exception as e:
        print(f"   -> Error: {e}")
        
    return data

# ==========================================
# 3. EXECUTION
# ==========================================
leagues = [
    ("Premier League", "England", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Premier_League"),
    ("La Liga", "Spain", "https://en.wikipedia.org/wiki/2024%E2%80%9325_La_Liga"),
    ("Bundesliga", "Germany", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Bundesliga"),
    ("Serie A", "Italy", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Serie_A"),
    ("Ligue 1", "France", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Ligue_1"),
    ("Primeira Liga", "Portugal", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Primeira_Liga"),
    ("Eredivisie", "Netherlands", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Eredivisie"),
    ("Belgian Pro League", "Belgium", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Belgian_Pro_League"),
    ("Süper Lig", "Turkey", "https://en.wikipedia.org/wiki/2024%E2%80%9325_S%C3%BCper_Lig"),
    ("Super League Greece", "Greece", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Super_League_Greece"),
    ("Austrian Bundesliga", "Austria", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Austrian_Football_Bundesliga"),
    ("Czech First League", "Czech Republic", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Czech_First_League"),
    ("Danish Superliga", "Denmark", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Danish_Superliga"),
    ("Polish Ekstraklasa", "Poland", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Ekstraklasa"),
    ("Ukrainian Premier League", "Ukraine", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Ukrainian_Premier_League"),
    ("Croatian Football League", "Croatia", "https://en.wikipedia.org/wiki/2024%E2%80%9325_Croatian_Football_League")
]

all_data = []
for l in leagues:
    all_data.extend(scrape_league(*l))

# Create DataFrame
df = pd.DataFrame(all_data)

# Filter out rows where "Team" is accidentally a header
if not df.empty:
    df = df[~df['Team'].str.contains("Stadium|Capacity", case=False, na=False)]

# Create the final clean dataset (Only rows with valid Lat/Lon)
df_stadiums = df.dropna(subset=['Latitude', 'Longitude']).copy()

print("\n" + "="*40)
print(f"Final Count: {len(df_stadiums)} stadiums ready.")
print("="*40)



Scraping: Premier League...
   -> Success. Found 20 teams. (20 with coords)
Scraping: La Liga...
   -> Success. Found 20 teams. (20 with coords)
Scraping: Bundesliga...
   -> Success. Found 18 teams. (18 with coords)
Scraping: Serie A...
   -> Success. Found 20 teams. (20 with coords)
Scraping: Ligue 1...
   -> Success. Found 18 teams. (18 with coords)
Scraping: Primeira Liga...
   -> Success. Found 18 teams. (18 with coords)
Scraping: Eredivisie...
   -> Success. Found 18 teams. (18 with coords)
Scraping: Belgian Pro League...
   -> Success. Found 16 teams. (16 with coords)
Scraping: Süper Lig...
   -> Success. Found 19 teams. (17 with coords)
Scraping: Super League Greece...
   -> Success. Found 14 teams. (14 with coords)
Scraping: Austrian Bundesliga...
   -> Success. Found 12 teams. (12 with coords)
Scraping: Czech First League...
   -> Success. Found 16 teams. (16 with coords)
Scraping: Danish Superliga...
   -> Success. Found 12 teams. (12 with coords)
Scraping: Polish Ekstraklas

In [0]:
display(df_stadiums.head(5))

League,Country,Team,Stadium,Latitude,Longitude
Premier League,England,Arsenal,Emirates Stadium,51.55667,-0.10611
Premier League,England,Aston Villa,Villa Park,52.509166666667,-1.8847222222222
Premier League,England,Bournemouth,Dean Court,50.73528,-1.83833
Premier League,England,Brentford,Brentford Community Stadium,51.49083,-0.28861
Premier League,England,Brighton & Hove Albion,Falmer Stadium,50.861822222222,-0.083277777777778


### 📍B.  Stadium URL Discovery (Web Scraping)
- **Source**: StadiumGuide.com (17 Major European Nations).
- **Goal**: To build a comprehensive index of valid stadium profile URLs for every major European football nation, serving as the foundation for the geolocation features.
- **Method**: We utilize requests with resilient proxy rotation (BrightData) to bypass anti-bot measures. The pipeline employs a "Country-First" strategy, crawling national overview pages rather than league tables to capture a static, complete list of stadiums. It applies robust filtering logic to strip out navigation links (e.g., "tickets", "about") and duplicates, ensuring only valid stadium profile links are retained for the final scraping phase.

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re
import urllib3
import warnings

# ==========================================
# 1. CONFIGURATION & PROXIES
# ==========================================

# Suppress SSL warnings
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.filterwarnings("ignore")

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/120.0.0.0 Safari/537.36'
}

PASSWORD = os.getenv('BRIGHTDATA_PASSWORD', 'YOUR_PASSWORD_HERE')
BASE_USERNAME = os.getenv('BRIGHTDATA_USER', 'YOUR_ZONE_USERNAME_HERE') 
HOST = 'brd.superproxy.io'
PORT = '33335'
country_code = "hu" 
session_id = random.randint(1, 1000000)
FINAL_USERNAME = f"{BASE_USERNAME}-country-{country_code}-session-{session_id}"

PROXIES = {
    'http': f'http://{FINAL_USERNAME}:{PASSWORD}@{HOST}:{PORT}', 
    'https': f'http://{FINAL_USERNAME}:{PASSWORD}@{HOST}:{PORT}'
}

# --- NEW: COUNTRY MAP ---
# We point to the main country index pages which are more reliable than league pages.
COUNTRY_MAP = [
    ("England", "https://www.stadiumguide.com/england/"),
    ("Spain", "https://www.stadiumguide.com/spain/"),
    ("Germany", "https://www.stadiumguide.com/germany/"),
    ("Italy", "https://www.stadiumguide.com/italy/"),
    ("France", "https://www.stadiumguide.com/france/"),
    ("Portugal", "https://www.stadiumguide.com/portugal/"),
    ("Netherlands", "https://www.stadiumguide.com/netherlands/"),
    ("Belgium", "https://www.stadiumguide.com/belgium/"),
    ("Turkey", "https://www.stadiumguide.com/turkey/"),
    ("Greece", "https://www.stadiumguide.com/greece/"),
    ("Denmark", "https://www.stadiumguide.com/denmark/"),
    ("Austria", "https://www.stadiumguide.com/austria/"),
    ("Poland", "https://www.stadiumguide.com/poland/"),
    ("Ukraine", "https://www.stadiumguide.com/ukraine/"),
    ("Croatia", "https://www.stadiumguide.com/croatia/"),
    ("Russia", "https://www.stadiumguide.com/russia/"),
    ("Scotland", "https://www.stadiumguide.com/scotland/")
]

# ==========================================
# 2. ROBUST SCRAPER FUNCTIONS
# ==========================================

def get_soup_with_retry(url, retries=3):
    """Tries to fetch the URL multiple times."""
    for i in range(retries):
        try:
            r = requests.get(url, proxies=PROXIES, headers=HEADERS, verify=False, timeout=20)
            if r.status_code == 200:
                return BeautifulSoup(r.content, 'html.parser')
            elif r.status_code == 404:
                print(f"  [!] 404 Not Found: {url}")
                return None
        except Exception as e:
            print(f"  [~] Retry {i+1}/{retries} for {url} ({str(e)[:50]}...)")
            time.sleep(2)
    print(f"  [!] Failed to load {url} after {retries} attempts.")
    return None

def extract_links_generic(soup, country_name):
    """
    Scans the page for links that look like stadium profiles.
    """
    stadiums = []
    # Find all 'a' tags with an href
    links = soup.find_all('a', href=True)
    
    for link in links:
        href = link['href']
        text = link.get_text(strip=True)
        
        # FILTERS:
        # 1. Must be on stadiumguide.com
        # 2. Must NOT be a known non-stadium page
        # 3. Link text must be substantial (> 3 chars)
        if "stadiumguide.com" in href and len(text) > 3:
            
            # Bad keywords that indicate a navigation link, not a stadium
            bad_keywords = [
                "category", "tournaments", "tickets", "about", "contact", 
                "privacy", "city-guides", "stadium-database", "future-stadiums",
                "past-stadiums", "tournament-stadiums"
            ]
            
            if not any(x in href for x in bad_keywords):
                # Ensure we aren't linking back to a country/league list page
                if "stadiums" not in href and "/present/" not in href:
                    # Specific check to avoid linking to the country page itself (e.g. /italy/)
                    if href.strip('/').split('/')[-1] != country_name.lower():
                        stadiums.append({
                            "Country": country_name,
                            "Stadium_Name_Overview": text,
                            "Stadium_URL": href
                        })
    return stadiums

def scrape_country_overview(country_name, url):
    print(f"Scraping Country: {country_name}...")
    
    soup = get_soup_with_retry(url)
    if not soup: return []
    
    # STRATEGY 1: "Nuclear Option" - Scan the main content area
    # Country pages often use a 'main' tag or 'div.entry-content'
    main_area = soup.find('main') or soup.find('div', class_='entry-content') or soup.body
    
    links = extract_links_generic(main_area, country_name)
    
    # Deduplicate URLs found on the page
    unique_links = []
    seen = set()
    for l in links:
        if l['Stadium_URL'] not in seen:
            unique_links.append(l)
            seen.add(l['Stadium_URL'])
            
    print(f"  -> Found {len(unique_links)} stadiums in {country_name}.")
    return unique_links

# ==========================================
# 3. EXECUTION
# ==========================================

all_country_data = []

for country, url in COUNTRY_MAP:
    data = scrape_country_overview(country, url)
    all_country_data.extend(data)
    time.sleep(random.uniform(1.0, 2.0)) # Be polite

# Create Master DataFrame
df_country_pages = pd.DataFrame(all_country_data)

# Remove duplicates (sometimes stadiums appear in multiple lists)
if not df_country_pages.empty:
    df_country_pages = df_country_pages.drop_duplicates(subset=['Stadium_URL'])


Scraping Country: England...
  -> Found 66 stadiums in England.
Scraping Country: Spain...
  -> Found 48 stadiums in Spain.
Scraping Country: Germany...
  -> Found 43 stadiums in Germany.
Scraping Country: Italy...
  -> Found 37 stadiums in Italy.
Scraping Country: France...
  -> Found 42 stadiums in France.
Scraping Country: Portugal...
  -> Found 15 stadiums in Portugal.
Scraping Country: Netherlands...
  -> Found 25 stadiums in Netherlands.
Scraping Country: Belgium...
  -> Found 16 stadiums in Belgium.
Scraping Country: Turkey...
  -> Found 14 stadiums in Turkey.
Scraping Country: Greece...
  -> Found 8 stadiums in Greece.
Scraping Country: Denmark...
  -> Found 9 stadiums in Denmark.
Scraping Country: Austria...
  -> Found 12 stadiums in Austria.
Scraping Country: Poland...
  -> Found 20 stadiums in Poland.
Scraping Country: Ukraine...
  -> Found 10 stadiums in Ukraine.
Scraping Country: Croatia...
  -> Found 6 stadiums in Croatia.
Scraping Country: Russia...
  -> Found 22 stadium

In [0]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
import random
import re

# ==========================================
# 1. SETUP & PROXIES
# ==========================================
# (Same configuration as before)
PASSWORD = os.getenv('BRIGHTDATA_PASSWORD', 'YOUR_PASSWORD_HERE')
BASE_USERNAME = os.getenv('BRIGHTDATA_USER', 'YOUR_ZONE_USERNAME_HERE') 
HOST = 'brd.superproxy.io'
PORT = '33335'
country_code = "hu" 
session_id = random.randint(1, 1000000)
FINAL_USERNAME = f"{BASE_USERNAME}-country-{country_code}-session-{session_id}"

PROXIES = {
    'http': f'http://{FINAL_USERNAME}:{PASSWORD}@{HOST}:{PORT}', 
    'https': f'http://{FINAL_USERNAME}:{PASSWORD}@{HOST}:{PORT}'
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36'
}

# ==========================================
# 2. HELPER FUNCTIONS (TEXT EXTRACTION)
# ==========================================

def clean_text(text):
    """Removes extra whitespace and newlines."""
    if not text: return "Info not available"
    return re.sub(r'\s+', ' ', text).strip()

def get_section_text(soup, patterns):
    """
    Scans for headers matching specific keywords (Directions, Tickets, etc.)
    and extracts the paragraphs following them.
    """
    header = None
    # Search for header tags (h2-h5, strong) matching patterns
    for tag in ['h2', 'h3', 'h4', 'h5', 'strong']: 
        for pattern in patterns:
            header = soup.find(tag, string=re.compile(pattern, re.IGNORECASE))
            if header: break
        if header: break
    
    if not header: return "Info not available"

    content = []
    curr = header.find_next_sibling()
    while curr:
        # Stop reading if we hit the next major header or footer
        if curr.name in ['h2', 'h3', 'div', 'footer']:
             # Only stop if the header actually has text (avoids empty tags stopping the scraper)
            if curr.get_text(strip=True): break
        
        # Collect text from paragraphs and lists
        if curr.name in ['p', 'ul', 'ol']:
            text = curr.get_text(strip=True)
            if len(text) > 5: content.append(text)
        
        curr = curr.find_next_sibling()
        
    return clean_text(" ".join(content)) if content else "Info not available"

def parse_stadium_page(url, country):
    """
    Visits a single stadium URL and extracts ALL data fields.
    """
    data = {
        "Country": country,
        "Stadium": None,
        "City": "Info not available",
        "Home Team": "Info not available",
        "Capacity": None,
        "Directions": "Info not available",
        "Food_and_Stay": "Info not available",
        "Ticket_Info": "Info not available",
        "URL": url
    }
    
    try:
        r = requests.get(url, proxies=PROXIES, headers=HEADERS, verify=False, timeout=15)
        if r.status_code != 200: return data 
        
        soup = BeautifulSoup(r.content, 'html.parser')
        
        # --- 1. BASIC INFO ---
        # Stadium Name
        title = soup.find('h1')
        data['Stadium'] = title.get_text(strip=True) if title else "Unknown"

        # Parsing the 'Stadfacts' table (common on StadiumGuide)
        facts_table = soup.find('table', class_='stadfacts')
        if facts_table:
            for row in facts_table.find_all('tr'):
                cells = row.find_all('td')
                if len(cells) >= 2:
                    key = cells[0].get_text(strip=True).lower()
                    val = cells[1].get_text(strip=True)
                    
                    if "club" in key or "team" in key:
                        data["Home Team"] = val
                    elif "location" in key or "city" in key:
                        data["City"] = val
                    elif "capacity" in key:
                        # Clean "75,000" -> 75000
                        clean_cap = re.sub(r'[^\d]', '', val)
                        if clean_cap: data["Capacity"] = int(clean_cap)
        
        # Fallback: If table is missing, use Regex on the body text
        content_div = soup.find('div', class_='entry-content')
        content_text = content_div.get_text(" ", strip=True) if content_div else ""

        if data["Home Team"] == "Info not available":
            match = re.search(r'(?:Team|Club):\s*([A-Za-z0-9\s]+)', content_text)
            if match: data["Home Team"] = match.group(1).strip()
            
        if not data["Capacity"]:
            match = re.search(r'Capacity:\s*([0-9,]+)', content_text)
            if match:
                clean_cap = re.sub(r'[^\d]', '', match.group(1))
                if clean_cap: data["Capacity"] = int(clean_cap)

        # --- 2. RICH TEXT SECTIONS ---
        # Regex patterns to capture variations in headings
        d_pats = [r"How to get to", r"Getting to", r"Transport", r"Location", r"Directions", r"Arriving"]
        f_pats = [r"Eat, drink", r"Food", r"Restaurants", r"Hotels", r"Sleep", r"Where to stay"]
        t_pats = [r"Tickets", r"Admissions", r"Buying tickets", r"Entry"]

        data['Directions'] = get_section_text(soup, d_pats)
        data['Food_and_Stay'] = get_section_text(soup, f_pats)
        data['Ticket_Info'] = get_section_text(soup, t_pats)

    except Exception as e:
        print(f"Error parsing {url}: {e}")
        
    return data

# ==========================================
# 3. MAIN EXECUTION LOOP
# ==========================================

print(f"Starting deep scrape for {len(df_country_pages)} stadiums...")
full_stadium_data = []

# Convert DataFrame to list of dicts for faster iteration
stadium_list = df_country_pages.to_dict('records')

for i, row in enumerate(stadium_list):
    url = row['Stadium_URL']
    cntry = row['Country']
    
    # Progress Bar
    print(f"[{i+1}/{len(stadium_list)}] Scraping: {row['Stadium_Name_Overview']}...", end="\r")
    
    # Scrape
    details = parse_stadium_page(url, cntry)
    full_stadium_data.append(details)
    
    # Polite sleep
    time.sleep(random.uniform(0.5, 1.2))

# ==========================================
# 4. FINAL OUTPUT
# ==========================================

df_stadiums_enriched = pd.DataFrame(full_stadium_data)

# Reorder columns nicely
cols = ['Country', 'Stadium', 'Directions', 'Food_and_Stay', 'Ticket_Info']
        
# Select only columns that exist
final_cols = [c for c in cols if c in df_stadiums_enriched.columns]
df_stadiums_enriched = df_stadiums_enriched[final_cols]

print("\n\n" + "="*40)
print(f"Scraping Complete! Collected data for {len(df_stadiums_enriched)} stadiums.")
print("="*40)



Starting deep scrape for 409 stadiums...
[409/409] Scraping: McDiarmid Park...

Scraping Complete! Collected data for 409 stadiums.


In [0]:
display(df_stadiums_enriched.head(5))

Country,Stadium,Directions,Food_and_Stay,Ticket_Info
England,Oakwell Stadium,"Oakwell Stadium is located just east of Barnsley’s town centre and rail station. The walk will only take 5 to 10 minutes. Barnsley is serviced with directtrainsfrom, among others, Sheffield, Leeds, and Huddersfield. Address: Grove Street, Barnsley, S71 1ET",Info not available,"Tickets for Barnsley matches can be boughtonline, or at the ticket office at the stadium. Tickets can also be bought at the turnstiles on the day of the match. Ticket prices are the same for all stands, but depend on the opponent. The cheapest go for £23.00, while tickets for the most expensive opponents cost £36.00. Tickets are £2.00 more expensive if bought on the day. For more information emailboxoffice@barnsleyfc.co.ukor call +44 (0)871 2266777."
England,Villa Park,Info not available,"Villa Park is located in a typical English urban area. This means that there are a few pubs around as well as some easy eating options, though if you have more time on your hands, Birmingham’s city centre might be a better option. If you arrive by car, you could stop at theStar Cityshopping centre just off exit 6 of the M6. It has the typical food outlets you tend to find at a shopping centre as well as various entertainment options. The nightlife in Birmingham’s city centre is mostly located west and south of Birmingham New Street Station. There are few hotels in the immediate vicinity of the stadium. TheHoliday Inn Expresslocated across Star City shopping centre is probably closest, but is mostly convenient if you get in by car. You can about just walk to the stadium from there. Further toward the centre, there are aCampanile HotelandPremier Inn. They are affordable and get good reviews, but are again most of all convenient for those arriving by car. If travelling by public transport, it is likely best to find a hotel in Birmingham’s centre, and take a bus or train to the match. For all hotels near Villa Park clickhere. For all options in the city centre gohere.","Tickets for Aston Villa matches can be boughtonline, by phone +44 (0) 800 6120 970, or at the Villa Village store at Villa Park. Tickets can also be bought at the same store before the start of the match. Aston Villa only very occasionally sell out. Aston Villa have divided their home games in three pricing categories. Tickets for category A matches, the most expensive, range in price from £25.00 for a lower-tier seat at the North Stand to £45.00 for a central seat at one of the long sides. Tickets for category VV matches, the cheapest, range in price from £20.00 to £37.00."
England,St. Andrew’s Stadium,Info not available,"St. Andrew’s Stadium is located in a rather quiet residential area with little around. As usual in England, there is always the odd pub, though you find many more options in Birmingham’s city centre, which is not far away. There are, for example, various eating and drinking options south of Birmingham New Street Station, from where one can walk or catch a bus to the ground. There is anIbis Hotela few minutes walking from St. Andrew’s, whileRoyal George Hotellies right at the back of the stadium. Both are reasonably priced and get acceptable reviews.The Moseley Armsis a similar alternative located slightly more toward the city centre. Clickherefor all hotels near St. Andrew’s Stadium. If you have a little more time on your hands, staying in Birmingham’s city centre may be a better alternative though. In particular from one of the hotels just south of BirminghamNew Street Stationit won’t be too hard to get to the stadium.","Tickets for Birmingham City games can be purchasedonline, by phone +44 (0) 844 557 1875, or in person at the Ticket Office at St. Andrew’s Stadium. If tickets remain, these can also be bought on the day of the match at the stadium. Birmingham City rarely sells out in the Championship. Ticket prices typically range from £25.00 for an upper-tier seat behind the goal to £30.00 for a seat at one of 

In [0]:
import pandas as pd
import unicodedata
import difflib
import re

# ==========================================
# 1. LOAD DATASETS
# ==========================================
print("Loading files...")

# Load from existing DataFrames
df_coords = df_stadiums.toPandas() if hasattr(df_stadiums, 'toPandas') else df_stadiums.copy()
df_details = df_stadiums_enriched.toPandas() if hasattr(df_stadiums_enriched, 'toPandas') else df_stadiums_enriched.copy()

print(f"Coords: {len(df_coords)} | Details: {len(df_details)}")

# ==========================================
# 2. ADVANCED NORMALIZATION
# ==========================================
def normalize(s):
    if pd.isna(s): return ""
    s = str(s).lower()
    s = re.sub(r'\[.*?\]', '', s)
    s = re.sub(r'\(.*?\)', '', s)
    s = s.replace("'", "'").replace("'", "'").replace("-", " ")
    s = unicodedata.normalize('NFKD', s).encode('ascii', 'ignore').decode('utf-8')
    return s.strip()

df_coords['norm_stad'] = df_coords['Stadium'].apply(normalize)
df_coords['norm_team'] = df_coords['Team'].apply(normalize)
df_coords['norm_country'] = df_coords['Country'].apply(normalize)

country_fix = {'the netherlands': 'netherlands', 'turkiye': 'turkey', 'czechia': 'czech republic'}

stad_col = next((c for c in df_details.columns if 'stadium' in c.lower()), 'Stadium')
cntry_col = next((c for c in df_details.columns if 'country' in c.lower()), 'Country')
url_col = next((c for c in df_details.columns if 'url' in c.lower()), None)
text_col = next((c for c in df_details.columns if 'directions' in c.lower()), 'Directions')

df_details['norm_stad'] = df_details[stad_col].apply(normalize)
df_details['norm_country'] = df_details[cntry_col].apply(lambda x: country_fix.get(normalize(x), normalize(x)))
df_details['clean_url'] = df_details[url_col].astype(str).str.lower() if url_col else ''
df_details['search_text'] = df_details[text_col].astype(str).str.lower() if text_col in df_details.columns else ''

details_list = df_details.to_dict('records')

# ==========================================
# 3. MANUAL MAP
# ==========================================
manual_map = {
    # --- SPAIN ---
    "anoeta": "reale arena",
    "balaidos": "balaidos",
    "mendizorrotza": "mendizorroza",
    "san mames": "san mames",
    "metropolitano": "wanda",
    "olimpic lluis companys": "olimpic",
    "ramon sanchez pizjuan": "sanchez pizjuan",
    "benito villamarin": "benito villamarin",
    "mestalla": "mestalla",
    "coliseum": "alfonso perez",
    "son moix": "iberostar",
    "mallorca son moix": "iberostar",
    
    # --- ITALY ---
    "unipol domus": "cagliari",
    "gewiss stadium": "atleti azzurri", 
    "mapei stadium": "citta del tricolore",
    "dacia arena": "friuli",
    "bluenergy stadium": "friuli",
    "stadio diego armando maradona": "san paolo",
    
    # --- NETHERLANDS ---
    "de adelaarshorst": "go ahead",
    "johan cruijff arena": "amsterdam arena",
    "mac3park stadion": "ijsseldelta",
    
    # --- BELGIUM ---
    "afas stadion achter de kazerne": "mechelen",
    "het kuipje": "westerlo",
    "lotto park": "vanden stock",
    "cegeka arena": "luminus",
    "planet group arena": "ghelamco",
    "den dreef": "den dreef",
    
    # --- TURKEY ---
    "corendon airlines park": "antalya",
    "konya metropolitan municipality stadium": "torku",
    "papara park": "senol gunes",
    "rams park": "turk telekom",
    "mersin stadium": "mersin",
    "tupras stadyumu": "vodafone",
    "ulker stadyumu": "sukru",
    "medas konya": "torku",
    "rize city stadium": "caykur didi",
    "samsun 19 mayis stadium": "samsun 19 mayis",
    "new adana stadium": "new adana",
    "gursel aksel stadium": "gursel aksel",
    "rhg enerturk enerji stadium": "kadir has",

    # --- AUSTRIA ---
    "profertil arena hartberg": "hartberg",
    "stadion schnabelholz": "cashpoint",
    "hofmann personal stadion": "donaupark",
    "raiffeisen arena": "linzer",

    # --- CZECH REPUBLIC ---
    "dolicek": "dolicek",
    "mestsky fotbalovy stadion miroslava valenty": "miroslava valenty",
    "mestsky stadion karvina": "karvina",
    "mestsky stadion ostrava": "mestsky stadion",
    "na stinadlech": "na stinadlech",

    # --- DENMARK ---
    "ceres park": "atletion",
    "energi viborg arena": "viborg",
    "right to dream park": "farum",

    # --- POLAND ---
    "jozef pilsudski cracovia stadium": "cracovia",
    "jozef pilsudski cracovia stadium3municipal stadium": "cracovia",
    "gks katowice stadium1arena katowice": "gks katowice",
    "ernest pohl stadium": "ernest pohl",
    "exbud arena": "kielce city",
    "polish army stadium": "polish army",
    "motor lublin arena": "arena lublin",
    "piotr wieczorek stadium": "piast",
    "florian krygier stadium": "florian krygier",
    "czachor brothers stadium": "radomiak",
    "rakow municipal football stadium": "rakow",
    "grzegorz lato stadium": "stal mielec",
    "widzew lodz stadium": "widzew",
    "kghm zaglebie arena": "zaglebie",
    "polsat plus arena gdansk": "pge arena",
    "tarczynski arena": "wroclaw",
    "pge narodowy": "narodowy",

    # --- UKRAINE ---
    "lobanovskyi dynamo stadium": "dynamo stadium",
    "berezkin zirka stadium": "zirka",
    "butovsky vorskla stadium": "vorskla",
    "avanhard stadium": "avanhard",
    "nsc olimpiyskiy": "olimpiyskiy",
    "csc nika stadium": "csc nika",

    # --- CROATIA ---
    "src velika gorica": "radnik",
    "kranjceviceva1": "kranjceviceva",
    "subicevac": "sibenik",
    "opus arena": "gradski vrt", 
    
    # --- PREMIER LEAGUE ---
    "city of manchester stadium": "etihad",
    "falmer stadium": "amex",
    "dean court": "vitality",
    "london stadium": "london stadium",
    "tottenham hotspur stadium": "tottenham",
    "brentford community stadium": "brentford"
}

# ==========================================
# 4. MATCHING LOGIC
# ==========================================
def get_match(row):
    target_stad = row['norm_stad']
    target_team = row['norm_team']
    target_country = row['norm_country']
    
    candidates = [d for d in details_list if d['norm_country'] == target_country]
    if not candidates: 
        candidates = details_list

    # 1. Exact Stadium
    for d in candidates:
        if d['norm_stad'] == target_stad: 
            return d

    # 2. Manual Map
    if target_stad in manual_map:
        key = manual_map[target_stad]
        for d in candidates:
            if key in d['norm_stad'] or key in d.get('clean_url', ''): 
                return d

    # 3. Team in URL
    if len(target_team) > 3:
        for d in candidates:
            if target_team in d.get('clean_url', ''): 
                return d

    # 4. Team in Text
    if len(target_team) > 4:
        for d in candidates:
            if target_team in d.get('search_text', ''): 
                return d

    # 5. Fuzzy Match
    all_stads = [d['norm_stad'] for d in candidates]
    matches = difflib.get_close_matches(target_stad, all_stads, n=1, cutoff=0.55)
    if matches:
        best = matches[0]
        for d in candidates:
            if d['norm_stad'] == best: 
                return d

    return None

# ==========================================
# 5. EXECUTE JOIN (LEFT JOIN - keep ALL df_coords rows)
# ==========================================
print("Joining...")
enriched = []
matched_count = 0
missing_count = 0

for idx, row in df_coords.iterrows():
    match = get_match(row)
    
    # Always create a row - LEFT JOIN behavior (keeps ALL df_coords rows)
    new_row = {
        'League': row['League'],
        'Country': row['Country'],
        'Team': row['Team'],
        'Stadium': row['Stadium'],
        'Latitude': row['Latitude'],
        'Longitude': row['Longitude'],
        'Directions': match.get('Directions') if match else None,
        'Food_and_Stay': match.get('Food_and_Stay') if match else None,
        'Ticket_Info': match.get('Ticket_Info') if match else None
    }
    enriched.append(new_row)
    
    if match:
        matched_count += 1
    else:
        missing_count += 1

df_final_stadiums = pd.DataFrame(enriched)

# ==========================================
# 6. OUTPUT
# ==========================================
print(f"Total: {len(df_final_stadiums)} | Matched: {matched_count} | Missing: {missing_count}")


save_path = "/dbfs/FileStore/Stadium_LLM_Enrichment.csv"
df_final_stadiums.to_csv(save_path, index=False)
print(f"Saved successfully to: {save_path}")

Loading files...
Coords: 274 | Details: 409
Joining...
Total: 274 | Matched: 243 | Missing: 31
Saved successfully to: /dbfs/FileStore/Stadium_LLM_Enrichment.csv


In [0]:
display(df_final_stadiums.head(5))


League,Country,Team,Stadium,Latitude,Longitude,Directions,Food_and_Stay,Ticket_Info
Premier League,England,Arsenal,Emirates Stadium,51.55667,-0.10611,"The Emirates Stadium is located in the Islington area, toward the north of London at just over 2 miles from Kings Cross St Pancras railway station. There are multiple ways to reach the stadium by public transport. Theunderground(tube) is one option – the nearest tube station is Arsenal, which is on the Piccadilly line. Decent alternatives are stations Finsbury Park (Victoria and Piccadilly line) and Highbury & Islington (Victoria line and London overground). From both stations it is an approximate 10-minute walk to the stadium. On non-matchdays, Holloway Road station (Piccadilly line) is located closest, but will be closed pre-match and is exit-only after the match. Alternatively, one can catch atrainto Finsbury Park or Highbury & Islington main line stations. It is a 5 to 10-minute journey coming from Kings Cross station. During the week trains depart from Moorgate as well. Drayton Park rail station, closest to the stadium, closes on matchdays. Supporters are not advised to arrive by car on matchdays, and there is little parking available around the Emirates Stadium on non-matchdays. Address: Emirates Stadium, London N5 1BU","The Emirates Stadium is located in the recently regenerated and rather quiet Highbury area. There are a few pubs and cafés in the immediate surroundings of the stadium, which is mainly residential, and some options to eat and drink along Holloway Road and Highbury Park road. More nightlife can be found in the Islington area on Upper Street starting south of Highbury & Islington tube station – an approximate 15 to 20-minute walk from the stadium. There are no hotels right near the Emirates Stadium, but a fair few options further north on Seven Sisters Road near Finsbury Park, which is a 15-minute walk from the stadium. TheQueens HotelandBest Western Highburyget the best reviews and go for about £100 a night, whereas thePembury Hotel,Woodberry Down Hotel, andCentral Park Hotelare more basic options. Clickherefor all hotels near the Emirates Stadium. Of course, as there are various tube lines passing by the stadium, you can just as well choose a stadium in any part ofCentral London.","Tickets for Arsenal matches can be boughtonline, or by phone 0844 277 3625 (+44 207 649 9003 if calling from abroad). Arsenal is one of the clubs in the Premier League for which it is hardest to get tickets and one will often need an Arsenal membership to be able to acquire tickets. The most basic Red level membership currently costs £33.00 and with it tickets will generally be available in pre-sale. Arsenal matches fall into three pricing categories. Tickets for category C matches, the cheapest, start at £26.00 for most lower-tier seats and range up to £51.00 for a central upper-tier seat. Tickets for category A matches, the most expensive, range from £63.50 for most lower-tier seats to £126.00 for a central upper-tier seat. A further booking fee of about £2.00 applies. Tickets for members are a little bit cheaper."
Premier League,England,Aston Villa,Villa Park,52.509166666667,-1.8847222222222,Info not available,"Villa Park is located in a typical English urban area. This means that there are a few pubs around as well as some easy eating options, though if you have more time on your hands, Birmingham’s city centre might be a better option. If you arrive by car, you could stop at theStar Cityshopping centre just off exit 6 of the M6. It has the typical food outlets you tend to find at a shopping centre as well as various entertainment options. The nightlife in Birmingham’s city centre is mostly located west and south of Birmingham New Street Station. There are few hotels in the immediate vicinity of the stadium. TheHoliday Inn Expresslocated across Star City shopping centre is probably closest, but is mostly convenient if you get in by car. You can about just walk to the stadium

### 📅 C. Football Match Schedule Ingestion (Web Scraping & APIs)
- **Source**: football-data.co.uk (15 Domestic Leagues) & Wikipedia (UEFA Competitions).
- **Goal**: To build a structured, unified table of historical and upcoming football matches (Dates, Teams, Leagues) that serves as the primary temporal filter for the recommendation engine.
- **Method**: The pipeline utilizes pd.read_csv to ingest structured historical data and BeautifulSoup to scrape unstructured HTML fixtures. It normalizes disparate date formats and team names into a standardized Delta table (football_matches_final) for downstream processing.
- **Domestic Leagues:** We ingest season-specific CSV feeds from *FixtureDownload.com*—spanning both the 2025/26 and 2026/27 campaigns—for 16 major competitions, including the "Big 5" (EPL, La Liga, etc.) and top tiers in the Netherlands, Portugal, Turkey, and Eastern Europe.
- **Champions League:** A targeted `BeautifulSoup` scraper parses Wikipedia to extract upcoming matches for both the 2025/26 Knockout Phase and the start of the 2026/27 League Phase.
- **Integration:** The pipeline strictly filters for 2026 dates, injects country metadata, normalizes timestamps to `YYYY-MM-DD`, and updates the **Spark Table** using schema-overwrite mode to ensure clean data persistence.

In [0]:
import pandas as pd
import requests
from io import StringIO
from bs4 import BeautifulSoup
import re
import random
from datetime import datetime, timedelta
import urllib3
import warnings

# ==========================================
# 0. CONFIGURATION & SETUP
# ==========================================
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
warnings.filterwarnings("ignore")

# Define Start Date (Adjust as needed)
START_DATE = datetime(2023, 8, 1)  

all_matches = []

# Proxy Configuration (User Provided)
PASSWORD = os.getenv('BRIGHTDATA_PASSWORD', 'YOUR_PASSWORD_HERE')
BASE_USERNAME = os.getenv('BRIGHTDATA_USER', 'YOUR_ZONE_USERNAME_HERE') 
HOST = 'brd.superproxy.io'
PORT = '33335'
country_code = "hu" 
session_id = random.randint(1, 1000000)
FINAL_USERNAME = f"{BASE_USERNAME}-country-{country_code}-session-{session_id}"

PROXIES = {
    'http': f'http://{FINAL_USERNAME}:{PASSWORD}@{HOST}:{PORT}', 
    'https': f'http://{FINAL_USERNAME}:{PASSWORD}@{HOST}:{PORT}'
}

HEADERS = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) '
                  'AppleWebKit/537.36 (KHTML, like Gecko) '
                  'Chrome/120.0.0.0 Safari/537.36'
}

# ==========================================
# PART 1: DOMESTIC LEAGUES (Source: Football-Data.co.uk)
# ==========================================
print("\n--- Step 1: Fetching Domestic Data (Football-Data.co.uk) ---")

# Mapping League Names to Football-Data.co.uk codes
# Note: 'football-data' covers major leagues deeply. Minor leagues are often in separate 'new' files.
league_map = {
    "Premier League": "E0",
    "Championship": "E1",
    "La Liga": "SP1",
    "Segunda Division": "SP2",
    "Bundesliga": "D1",
    "2. Bundesliga": "D2",
    "Serie A": "I1",
    "Serie B": "I2",
    "Ligue 1": "F1",
    "Ligue 2": "F2",
    "Eredivisie": "N1",
    "Belgian Pro League": "B1",
    "Primeira Liga": "P1",
    "Super Lig": "T1",
    "Super League Greece": "G1"
}

# Generate Season Strings for URLs (e.g., 2023 -> '2324')
# We scan a few years back to ensure we get the requested history
years = [2022, 2023, 2024, 2025] 
season_codes = []
for y in years:
    suffix = (y + 1) % 100
    code = f"{y % 100:02d}{suffix:02d}" # e.g., 2324
    season_codes.append(code)

base_url_template = "https://www.football-data.co.uk/mmz4281/{}/{}.csv"

for league_name, code in league_map.items():
    for season in season_codes:
        url = base_url_template.format(season, code)
        try:
            # Note: Verify=False is risky but kept per your config.
            response = requests.get(url, headers=HEADERS, proxies=PROXIES, verify=False, timeout=20)
            
            if response.status_code == 200:
                try:
                    pdf = pd.read_csv(StringIO(response.text))
                except:
                    # Fallback for common encoding issues in older CSVs
                    pdf = pd.read_csv(StringIO(response.text), encoding='latin1')

                # Normalize Columns to match your schema
                if 'Date' in pdf.columns and 'HomeTeam' in pdf.columns:
                    # Date parsing (football-data uses dd/mm/yy or dd/mm/yyyy)
                    pdf['DT_Obj'] = pd.to_datetime(pdf['Date'], dayfirst=True, errors='coerce')
                    pdf = pdf.dropna(subset=['DT_Obj']) # Drop rows with bad dates
                    
                    pdf['Date'] = pdf['DT_Obj'].dt.strftime('%Y-%m-%d')
                    
                    # Time is often present, but if missing default to 00:00
                    if 'Time' in pdf.columns:
                        pdf['Time'] = pdf['Time'].fillna('00:00').astype(str)
                    else:
                        pdf['Time'] = '00:00'

                    # Select & Rename
                    subset = pdf[['Date', 'Time', 'HomeTeam', 'AwayTeam', 'DT_Obj']].copy()
                    subset['League'] = league_name
                    
                    # Filter by Date
                    subset = subset[subset['DT_Obj'] >= START_DATE]
                    subset = subset.drop(columns=['DT_Obj'])

                    if not subset.empty:
                        all_matches.append(subset)
                        # print(f"   -> Fetched {league_name} ({season})")
        except Exception as e:
            # 404 is expected for future seasons that don't exist yet
            continue

print(f"   [INFO] Historical/Played matches processed. Total chunks: {len(all_matches)}")

# --- PART 1.5: FETCH UPCOMING FIXTURES (Next 1-2 Weeks) ---
# Football-Data provides a specific file for upcoming unplayed matches
try:
    url_fixtures = "https://www.football-data.co.uk/fixtures.csv"
    resp_fix = requests.get(url_fixtures, headers=HEADERS, proxies=PROXIES, verify=False, timeout=20)
    if resp_fix.status_code == 200:
        pdf_fix = pd.read_csv(StringIO(resp_fix.text), encoding='latin1')
        
        # In this file, 'Div' is the league code. We map it back to names.
        inv_map = {v: k for k, v in league_map.items()}
        pdf_fix['League'] = pdf_fix['Div'].map(inv_map)
        pdf_fix = pdf_fix.dropna(subset=['League']) # Keep only leagues we track
        
        pdf_fix['DT_Obj'] = pd.to_datetime(pdf_fix['Date'], dayfirst=True, errors='coerce')
        pdf_fix['Date'] = pdf_fix['DT_Obj'].dt.strftime('%Y-%m-%d')
        if 'Time' in pdf_fix.columns:
            pdf_fix['Time'] = pdf_fix['Time'].fillna('00:00')
        else:
            pdf_fix['Time'] = '00:00'
            
        subset_fix = pdf_fix[['Date', 'Time', 'HomeTeam', 'AwayTeam', 'League']].copy()
        
        # Only future dates
        subset_fix = subset_fix[pd.to_datetime(subset_fix['Date']) >= datetime.now()]
        
        if not subset_fix.empty:
            all_matches.append(subset_fix)
            print(f"   [INFO] Fetched {len(subset_fix)} upcoming fixtures from main feed.")
except Exception as e:
    print(f"   [WARN] Could not fetch upcoming fixtures file: {e}")


# ==========================================
# PART 2: EUROPEAN COMPETITIONS (Wikipedia)
# ==========================================
print("\n--- Step 2: Scraping European Competitions (Wikipedia) ---")

wiki_config = [
    # Champions League
    ("https://en.wikipedia.org/wiki/2023%E2%80%9324_UEFA_Champions_League_group_stage", "CL Group 23/24"),
    ("https://en.wikipedia.org/wiki/2023%E2%80%9324_UEFA_Champions_League_knockout_phase", "CL Knockout 23/24"),
    ("https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Champions_League_league_phase", "CL League 24/25"),
    ("https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Champions_League_knockout_phase", "CL Knockout 24/25"),
    # Europa League
    ("https://en.wikipedia.org/wiki/2023%E2%80%9324_UEFA_Europa_League_group_stage", "EL Group 23/24"),
    ("https://en.wikipedia.org/wiki/2023%E2%80%9324_UEFA_Europa_League_knockout_phase", "EL Knockout 23/24"),
    ("https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Europa_League_league_phase", "EL League 24/25"),
    ("https://en.wikipedia.org/wiki/2024%E2%80%9325_UEFA_Europa_League_knockout_phase", "EL Knockout 24/25")
]

for url, label in wiki_config:
    try:
        response = requests.get(url, headers=HEADERS, proxies=PROXIES, verify=False, timeout=30)
        soup = BeautifulSoup(response.content, 'html.parser')
        
        match_boxes = soup.find_all('div', class_='footballbox')
        count = 0

        for match in match_boxes: 
            try:
                # Robust extraction
                home_tag = match.find(class_='fhome') or match.find('th', class_='fhome')
                away_tag = match.find(class_='faway') or match.find('th', class_='faway')
                
                if not home_tag or not away_tag: continue
                
                home = home_tag.get_text(strip=True)
                away = away_tag.get_text(strip=True)
                
                date_div = match.find(class_='fdate')
                time_div = match.find(class_='ftime')
                raw_date = date_div.get_text(strip=True) if date_div else ""
                match_time = time_div.get_text(strip=True) if time_div else "00:00"

                # Date Parsing (Handles "25 October 2023")
                match_date_match = re.search(r'(\d{1,2}\s+\w+\s+\d{4})', raw_date)
                if match_date_match:
                    clean_date_str = match_date_match.group(1)
                    try:
                        dt_obj = datetime.strptime(clean_date_str, "%d %B %Y")
                    except ValueError:
                        continue # Skip bad dates
                    
                    if dt_obj >= START_DATE:
                        fmt_date = dt_obj.strftime("%Y-%m-%d")
                        league_label = "Champions League" if "Champions" in label else "Europa League"
                        
                        all_matches.append(pd.DataFrame([{
                            "Date": fmt_date, 
                            "Time": match_time,
                            "HomeTeam": home, 
                            "AwayTeam": away, 
                            "League": league_label,
                        }]))
                        count += 1
            except Exception as row_err: 
                continue
        print(f"   [OK] Processed {label:<20} - Found {count} matches")

    except Exception as e:
        print(f"   [ERROR] Scrape failed for {label}: {e}")

# ==========================================
# PART 3: CONSOLIDATE & SAVE
# ==========================================
print("\n--- Step 3: Consolidating Data ---")

if all_matches:
    full_schedule = pd.concat(all_matches, ignore_index=True)
    
    # Clean up whitespace
    full_schedule['HomeTeam'] = full_schedule['HomeTeam'].str.strip()
    full_schedule['AwayTeam'] = full_schedule['AwayTeam'].str.strip()
    
    # Sort and Drop Duplicates
    full_schedule = full_schedule.sort_values(by=['Date', 'Time'])
    full_schedule = full_schedule.drop_duplicates(subset=['Date', 'HomeTeam', 'AwayTeam'])
    
    print(f"   Total Matches Found: {len(full_schedule)}")

    
    # --- DATABRICKS / SPARK SAVE ---
    # Only runs if 'spark' session is available (Databricks env)
    try:
        if 'spark' in globals():
            df_schedule = spark.createDataFrame(full_schedule)
            df_schedule.write.mode("overwrite").option("overwriteSchema", "true").saveAsTable("football_matches_final")
            print("\n   [SUCCESS] Data saved to table 'football_matches_final'")
        else:
            print("\n   [NOTE] Spark not detected. Skipping table save.")
    except Exception as e:
        print(f"\n   [ERROR] Could not save to Spark: {e}")

else:
    print("\n   [WARNING] No matches found.")


--- Step 1: Fetching Domestic Data (Football-Data.co.uk) ---
   [INFO] Historical/Played matches processed. Total chunks: 45
   [WARN] Could not fetch upcoming fixtures file: 'Div'

--- Step 2: Scraping European Competitions (Wikipedia) ---
   [OK] Processed CL Group 23/24       - Found 96 matches
   [OK] Processed CL Knockout 23/24    - Found 29 matches
   [OK] Processed CL League 24/25      - Found 144 matches
   [OK] Processed CL Knockout 24/25    - Found 45 matches
   [OK] Processed EL Group 23/24       - Found 96 matches
   [OK] Processed EL Knockout 23/24    - Found 45 matches
   [OK] Processed EL League 24/25      - Found 144 matches
   [OK] Processed EL Knockout 24/25    - Found 45 matches

--- Step 3: Consolidating Data ---
   Total Matches Found: 14224

   [SUCCESS] Data saved to table 'football_matches_final'


In [0]:
display(df_schedule.limit(5))

Date,Time,HomeTeam,AwayTeam,League
2023-08-04,17:30,Hertha,Wehen,2. Bundesliga
2023-08-04,17:30,Paderborn,Osnabruck,2. Bundesliga
2023-08-04,19:45,Standard,St. Gilloise,Belgian Pro League
2023-08-04,20:00,Sheffield Weds,Southampton,Championship
2023-08-05,12:00,Elversberg,Hansa Rostock,2. Bundesliga


### 🔎D. Data Enrichment & Integration

This step merges the **Match Schedules** and **Stadium Locations** into a single master table.

* **The Challenge:** Discrepancies in team names between sources (e.g., *"Man United"* vs. *"Manchester United"*).
* **The Solution:** We implement a **normalization dictionary** to map CSV names to their Wikipedia counterparts. A smart join key (`coalesce`) ensures matches are linked correctly even when naming varies.
* **The Result:**  An inner join produces `euro_matches_enriched`, a clean dataset containing the **Who, When, and Where** for every match, ready for geospatial analysis.

In [0]:
import pandas as pd
import difflib
import unicodedata

# ---------------------------------------------------------
# 1. Helper to ensure standard Pandas DataFrames
# ---------------------------------------------------------
def ensure_local_pandas(df):
    if hasattr(df, 'toPandas'): return df.toPandas()
    elif hasattr(df, 'to_pandas'): return df.to_pandas()
    return df

# ---------------------------------------------------------
# 2. Load Data
# ---------------------------------------------------------
pdf_games = ensure_local_pandas(df_schedule).copy()
pdf_stadiums = ensure_local_pandas(df_final_stadiums).copy()

# ---------------------------------------------------------
# 3. Setup Logic
# ---------------------------------------------------------
def normalize_text(text):
    if not isinstance(text, str): return ""
    text = unicodedata.normalize('NFKD', text).encode('ascii', 'ignore').decode('utf-8')
    return text.lower().strip()

aliases = {
    'Man Utd': 'Manchester United', 'Man City': 'Manchester City',
    'Spurs': 'Tottenham Hotspur', 'Wolves': 'Wolverhampton Wanderers',
    'Leeds': 'Leeds United', 'Nottm Forest': 'Nottingham Forest',
    'Sheff Utd': 'Sheffield United', 'Stade Rennais FC': 'Rennes',
    'RC Celta': 'Celta Vigo', 'Cadiz': 'Cádiz CF',
    'Paris SG': 'Paris Saint-Germain', 'PSG': 'Paris Saint-Germain',
    'Sporting CP': 'Sporting', 'Inter': 'Internazionale', 
    'Milan': 'AC Milan', 'Athletic Club': 'Athletic Bilbao', 
    'Atletico': 'Atlético Madrid'
}

pdf_stadiums = pdf_stadiums.drop_duplicates(subset=['Team'])
stadium_teams_by_league = {}
for league, group in pdf_stadiums.groupby('League'):
    stadium_teams_by_league[league] = [(normalize_text(t), t) for t in group['Team'].unique()]
all_stadium_teams = [(normalize_text(t), t) for t in pdf_stadiums['Team'].unique()]

def find_best_match(row):
    team_name = row.get('HomeTeam')
    league = row.get('League')
    if pd.isna(team_name): return None
    
    if team_name in aliases:
        target = aliases[team_name]
        norm_target = normalize_text(target)
        for n, o in all_stadium_teams:
            if n == norm_target: return o
        return target 

    norm_name = normalize_text(team_name)
    candidates = stadium_teams_by_league.get(league, [])
    if league == 'Europa League' or not candidates: candidates = all_stadium_teams
        
    for n, o in candidates:
        if n == norm_name: return o
    for n, o in candidates:
        if (n in norm_name and len(n) > 3) or (norm_name in n and len(norm_name) > 3): return o
            
    cand_names = [c[0] for c in candidates]
    matches = difflib.get_close_matches(norm_name, cand_names, n=1, cutoff=0.6)
    if matches:
        for n, o in candidates:
            if n == matches[0]: return o
            
    if league != 'Europa League':
        for n, o in all_stadium_teams:
            if n == norm_name: return o
        matches = difflib.get_close_matches(norm_name, [c[0] for c in all_stadium_teams], n=1, cutoff=0.75)
        if matches:
            for n, o in all_stadium_teams:
                if n == matches[0]: return o
    return None

# ---------------------------------------------------------
# 4. Apply, Merge, and Filter
# ---------------------------------------------------------
print("Matching teams...")
pdf_games['StadiumTeam'] = pdf_games.apply(find_best_match, axis=1)

print("Merging...")
df_merged = pd.merge(
    pdf_games, 
    pdf_stadiums, 
    left_on='StadiumTeam', 
    right_on='Team', 
    how='left', 
    suffixes=('', '_stadium')
)

# --- FILTERING STEP ---
# Drop rows where 'Stadium' is NaN (null)
df_final = df_merged[df_merged['Stadium'].notna()]

# Cleanup columns
cols_to_drop = ['StadiumTeam', 'Team', 'League_stadium']
df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns], inplace=True)

# ---------------------------------------------------------
# 5. Header Standardization & Output
# ---------------------------------------------------------



# 2. Define the mapping from your CURRENT columns to the TARGET headers
# (Adjust the left-side keys if your stadium/schedule source names differ slightly)
rename_map = {
    'HomeTeam': 'home_team',
    'AwayTeam': 'away_team',
    'League': 'league_name',
    'Stadium': 'stadium_name',
    # Handle variations in stadium source data
    'Lat': 'latitude',       'Latitude': 'latitude',
    'Lon': 'longitude',      'Longitude': 'longitude'
}

# 3. Apply renaming
df_final = df_final.rename(columns=rename_map)

# 4. Define the strict final schema requested
target_headers = [
    "Date",
    "Time",
    "Country", 
    "home_team",
    "away_team",
    "league_name",
    "stadium_name",
    "latitude",
    "longitude"
]

# 5. Filter and Reorder columns
# This ensures only the requested columns exist in the final output
# using reindex to avoid errors if a column (like 'Country') is temporarily missing
df_final = df_final.reindex(columns=target_headers)

print(f"Final Filtered Count: {len(df_final)}")

# Use the full path explicitly
save_path = "/dbfs/FileStore/Match_Schedule_All_Leagues.csv"

print(f"Saving final data to {save_path}...")
df_final.to_csv(save_path, index=False)
print("Saved successfully.")

Matching teams...
Merging...
Final Filtered Count: 10264
Saving final data to /dbfs/FileStore/Match_Schedule_All_Leagues.csv...
Saved successfully.


In [0]:
display(df_final.head(5))

Date,Time,Country,home_team,away_team,league_name,stadium_name,latitude,longitude
2023-08-04,19:45,Belgium,Standard,St. Gilloise,Belgian Pro League,Stade Maurice Dufrasne,50.61,5.54333
2023-08-05,12:00,Germany,Holstein Kiel,Greuther Furth,2. Bundesliga,Holstein-Stadion,54.34917,10.12361
2023-08-05,12:00,Germany,St Pauli,Fortuna Dusseldorf,2. Bundesliga,Millerntor-Stadion,53.554583,9.967667
2023-08-05,14:00,France,St Etienne,Grenoble,Ligue 2,Stade Geoffroy Guichard,45.46083,4.39028
2023-08-05,15:00,England,Bristol City,Preston,Championship,King Power Stadium,52.620277777778,-1.1422222222222
